# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# read in the .csv
ct_data = "..\output_data\cities.csv"
city_df = pd.read_csv(ct_data)
del city_df['Unnamed: 0']
city_df[['Latitude' , 'Longitude']] = city_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
city_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Date of Weather
0,Albany,42.6001,-73.9662,67.87,76,17,4.56,09/11/2021
1,Cherskiy,68.7500,161.3000,33.28,54,96,1.79,09/11/2021
2,Maumere,-8.6199,122.2111,81.50,71,69,3.51,09/11/2021
3,Chokurdakh,70.6333,147.9167,44.02,55,85,15.32,09/11/2021
4,Luanda,-8.8368,13.2343,72.45,91,66,2.71,09/11/2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
import gmaps

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
# creating a new dataframe for only those cities that are between 70 and 80 degrees, windspeed less than 10
ideal_city = city_df.loc[(city_df['Tempature']>=70) & (city_df['Tempature'] <=80)]
ideal_city = ideal_city.loc[(ideal_city['Windspeed'] <= 10)]
ideal_city.dropna()
ideal_city.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Date of Weather
4,Luanda,-8.8368,13.2343,72.45,91,66,2.71,09/11/2021
18,Escanaba,45.7453,-87.0646,70.23,62,50,7.63,09/11/2021
21,Kaili,-3.4626,120.2147,71.37,90,85,1.54,09/11/2021
31,Vaini,-21.2000,-175.2000,73.56,94,90,8.05,09/11/2021
33,Ponta do Sol,32.6667,-17.1000,76.01,86,36,8.34,09/11/2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# geocoordinates
target_coordinates = "-8.8368, 13.2343"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [47]:
# convert the response to JSON
hotel_data = response.json()

# Print the json (pretty printed)
print(json.dumps(hotel_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "Aap_uECQhLPdf0rVX2ttxpb-cp2ww43wvcgq4i8wKKHHEj4l5vHhvOR3sOZPal1s8siIGt-0-IPUWtWTDm5cy4kBIoBkbYzG6nEpSq8N98DehqvNjObErproSwcY9_fNp5jKN6coTazxXyAYLy32vQKYvsjuUhgiRt2ZKA8F3Cqc6gWJ5bXoq-RKr4j2Ydu09yAFMxiyzjQVAIeLMkJhllHE02jhYQYaUOTlIYrMbcE7snRcxQyDbRitLAijsRvsxtPTz0wzUQ_vl5jNE9R8qRnWpLyjRDZFEqx3exgpWXvqKrOobVcICshCn035S6Qlhit_Du0sxTcWT4yd5g7knQgxdUX9FKGH2Q86qz-XfX0b81Bb6zPFz3dUT42KvPWJ7E3oJCE5gWeIoBbD_7tmMx0j8Yt3pdqb8GNFrww-H-fCQgpljDgUwK4m7A8_x6XalKY",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.8144567,
                    "lng": 13.2350337
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.813115220107276,
                        "lng": 13.23624852989272
                    },
                    "southwest": {
                        "lat": -8.815814879

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
